In [1]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 27 (delta 10), reused 23 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 6.89 KiB | 3.45 MiB/s, done.
Resolving deltas: 100% (10/10), done.
/kaggle/working/agnews-classifier


In [39]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=6,
    lora_alpha=18,
    lora_dropout=0.05,
    bias='none',
    target_modules=['query','key','value'],
    task_type='SEQ_CLS',
    use_rslora = False # regular LoRA: lora_alpha/r, rs-LoRA: lora_alpha/sqrt(r) -> normalization technique
)

model = create_lora_model(lora_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 925,444 || all params: 125,574,152 || trainable%: 0.7370


In [40]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): Mo

In [41]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=2000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='rmsprop',
    learning_rate=5e-5,
    lr_scheduler_type='cosine',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,0.749100,0.333113,0.890625
200,0.361600,0.332897,0.890625
300,0.332100,0.357226,0.887500
400,0.303400,0.327591,0.898438
500,0.299600,0.332403,0.896875
600,0.290200,0.314865,0.909375
700,0.294700,0.329329,0.900000
800,0.294200,0.312330,0.906250
900,0.287300,0.314165,0.909375
1000,0.294200,0.306584,0.906250


In [42]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-2000'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [00:59<00:00,  4.22it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-2000.csv


In [43]:
import os
import pandas as pd

# Define paths
csv_path = "/kaggle/working/saved_models/checkpoint-2000/log_history.csv"
output_dir = "/kaggle/working/processed_data"
output_csv = os.path.join(output_dir, "processed_log_history.csv")

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Read the csv file
df = pd.read_csv(csv_path)

# Select desired columns and reorder the dataframe
desired_order = [
    "step",         
    "loss",         
    "eval_loss",   
    "accuracy",     
    "eval_accuracy",
    "learning_rate",
    "epoch"         
]
df = df[desired_order]

# Rename columns for uniformity
df.rename(columns={
    "step": "Step",
    "loss": "Train Loss",
    "eval_loss": "Test Loss",
    "accuracy": "Train Acc",
    "eval_accuracy": "Test Acc",
    "learning_rate": "Learning Rate",
    "epoch": "Epochs"
}, inplace=True)

# Adding loss spread and loss ratio columns
df["loss spread"] = df["Train Loss"] - df["Test Loss"]
df["loss ratio"] = df["Train Loss"] / df["Test Loss"]

# Adding acc spread and acc ratio columns
df["Acc spread"] = df["Test Acc"] - df["Train Acc"]
df["Acc ratio"] = df["Test Acc"] / df["Train Acc"]

# Export the DataFrame as a csv file
df.to_csv(output_csv, index=False)

print(f"Data exported to {output_csv}")

df

Data exported to /kaggle/working/processed_data/processed_log_history.csv


,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,loss spread,loss ratio,Acc spread,Acc ratio
0,100,0.7491,0.333113,0.722500,0.890625,4.969221e-05,0.013405,0.415987,2.248788,0.168125,1.232699
1,200,0.3616,0.332897,0.883482,0.890625,4.877641e-05,0.026810,0.028703,1.086222,0.007143,1.008085
2,300,0.3321,0.357226,0.884821,0.887500,4.727516e-05,0.040214,-0.025126,0.929664,0.002679,1.003027
3,400,0.3034,0.327591,0.894196,0.898438,4.522542e-05,0.053619,-0.024191,0.926154,0.004241,1.004743
4,500,0.2996,0.332403,0.897768,0.896875,4.267767e-05,0.067024,-0.032803,0.901317,-0.000893,0.999005
5,600,0.2902,0.314865,0.899554,0.909375,3.969463e-05,0.080429,-0.024665,0.921665,0.009821,1.010918
6,700,0.2947,0.329329,0.907589,0.900000,3.634976e-05,0.093834,-0.034629,0.894850,-0.007589,0.991638
7,800,0.2942,0.312330,0.898214,0.906250,3.272542e-05,0.107239,-0.018130,0.941953,0.008036,1.008946
8,900,0.2873,0.314165,0.903571,0.909375,2.891086e-05,0.120643,-0.026865,0.914487,0.005804,1.006423
9,1000,0.2942,0.306584,0.900000,0.906250,2.500000e-05,0.134048,-0.012384,0.959605,0.006250,1.006944
